## Simple Notebook to evaluate the performances of linear decoding models

In [75]:
import numpy as np
import himalaya
import os
import torch
from himalaya.backend import set_backend
from os.path import join as opj
import pickle
from himalaya.ridge import RidgeCV
from sklearn.metrics.pairwise import cosine_similarity
import tqdm
import pandas as pd

In [76]:
# Identification accuracy function
def identification_accuracy_fast(P, T):
    n = P.shape[0]
    if n == 0:
        return np.nan

    P_mean = np.mean(P, axis=1, keepdims=True)
    T_mean = np.mean(T, axis=1, keepdims=True)
    P_std = np.std(P, axis=1, keepdims=True)
    T_std = np.std(T, axis=1, keepdims=True)
    
    P_normalized = (P - P_mean) / P_std
    T_normalized = (T - T_mean) / T_std
    C = np.dot(P_normalized, T_normalized.T) / P.shape[1]

    id_acc = np.zeros(n)
    for i in range(n):
        id_acc[i] = (C[i, i] >= C[i]).sum()  
        id_acc[i] = id_acc[i] / (n - 1)
    
    return np.mean(id_acc)


def evaluate_and_log(y_pred,y_gt,subject_id):
    
    
    y_pred_np = y_pred
    y_gt_np = y_gt

    # Compute similarity matrix
    similarity_matrix = cosine_similarity(y_pred_np, y_gt_np)

    # Calculate Top-1 and Top-5 accuracy
    top1_acc = 100*(np.argmax(similarity_matrix, axis=1) == np.arange(len(y_gt_np))).mean()
    top5_acc = 100*np.mean([1 if i in np.argsort(-similarity_matrix[i])[:5] else 0 for i in range(len(y_gt_np))])
    print(f"Top-1 Accuracy: {top1_acc:.4f}, Top-5 Accuracy: {top5_acc:.4f} ")

    # Identification accuracy
    id_acc = 100*identification_accuracy_fast(y_pred_np, y_gt_np)
    print(f"Identification accuracy: {id_acc:.4f}")

    baseline_top1 = 100/len(y_gt_np)
    baseline_top5 = 500/len(y_gt_np)

    improvement_top1 = top1_acc/baseline_top1
    improvement_top5 = top5_acc/baseline_top5
    # Baseline performance (50% for identification accuracy as requested)
    id_acc_baseline = 50
    results = { "Subject": subject_id,

        "Identification Accuracy (%)": id_acc,
        "ID Accuracy Baseline (%)": id_acc_baseline,
        "Top-1 Accuracy (%)": top1_acc,
        "Top1 Baseline (%)": baseline_top1,
        "Top1 Improvement Over Baseline": improvement_top1,
        "Top-5 Accuracy (%)": top5_acc,
        "Top5 Baseline (%)": baseline_top5,
        "Top5 Improvement Over Baseline": improvement_top5}
    return results


## Image linear decoding

In [ ]:
data_dir = "/home/matteo/storage/brain_tuning"

image_results = []
image_models = {}

for i, subj in tqdm.tqdm(enumerate(["CSI1","CSI2","CSI3","CSI4"]),total=4):

    #Load test fMRI data, test features and decoding model


    ## encode the augmented images using the image encoding model 
    device_id = 0
    torch.cuda.set_device(device_id)  # Set the current device

    device = f"cuda:{device_id}"
    backend = set_backend("torch_cuda")
    data_path =  f"/home/matteo/storage/brain_tuning/{subj}"

    test_fmri_top = np.load(opj(data_path, "test_fmri_top.npy"))
    test_features = np.load(opj(data_path, "test_image_features.npy"))
    train_fmri_top = np.load(opj(data_path, "train_fmri_top.npy"))
    train_features = np.load(opj(data_path, "train_image_features.npy"))

    simple_decoding_model = RidgeCV(alphas=[1, 10, 100, 1e3, 1e4])
    simple_decoding_model.fit(backend.asarray(train_fmri_top).to(device),backend.asarray(train_features).to(device))

    #Use the model to predict test set image features
    test_predictions = simple_decoding_model.predict(backend.asarray(test_fmri_top).to(device))



    res = evaluate_and_log(test_predictions.numpy(),test_features,i+1)
    image_results.append(res)

    image_models[subj] = {"model":simple_decoding_model, "test_predictions":test_predictions, "test_features":test_features}


  0%|          | 0/4 [00:00<?, ?it/s]/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 3795 < 10000). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(
 25%|██▌       | 1/4 [00:07<00:21,  7.14s/it]

Top-1 Accuracy: 7.7313, Top-5 Accuracy: 24.5881 
Identification accuracy: 94.1004


/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 3795 < 10000). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(
 50%|█████     | 2/4 [00:14<00:14,  7.15s/it]

Top-1 Accuracy: 4.8162, Top-5 Accuracy: 15.4626 
Identification accuracy: 91.5547


/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 3795 < 10000). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(
 75%|███████▌  | 3/4 [00:22<00:07,  7.64s/it]

Top-1 Accuracy: 5.1965, Top-5 Accuracy: 16.3498 
Identification accuracy: 91.5687


/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 2244 < 10000). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(
100%|██████████| 4/4 [00:25<00:00,  6.28s/it]

Top-1 Accuracy: 4.2827, Top-5 Accuracy: 17.7730 
Identification accuracy: 91.6061


In [78]:
df_image = pd.DataFrame.from_dict(image_results)
df_image.to_csv("/home/matteo/storage/brain_tuning/results_image_linear.csv")

In [79]:
df_image

,Subject,Identification Accuracy (%),ID Accuracy Baseline (%),Top-1 Accuracy (%),Top1 Baseline (%),Top1 Improvement Over Baseline,Top-5 Accuracy (%),Top5 Baseline (%),Top5 Improvement Over Baseline
0,0,94.100352,50,7.731305,0.126743,61.0,24.588086,0.633714,38.8
1,1,91.554721,50,4.816223,0.126743,38.0,15.462611,0.633714,24.4
2,2,91.568714,50,5.196451,0.126743,41.0,16.349810,0.633714,25.8
3,3,91.606088,50,4.282655,0.214133,20.0,17.773019,1.070664,16.6


## Text decoding evaluation

In [ ]:
data_dir = "/home/matteo/storage/brain_tuning"

text_results = []
text_models = {}

for i, subj in tqdm.tqdm(enumerate(["CSI1","CSI2","CSI3","CSI4"]),total=4):

    #Load test fMRI data, test features and decoding model


    ## encode the augmented images using the image encoding model 
    device_id = 0
    torch.cuda.set_device(device_id)  # Set the current device

    device = f"cuda:{device_id}"
    backend = set_backend("torch_cuda")
    data_path =  f"/home/matteo/storage/brain_tuning/{subj}"

    test_fmri_top = np.load(opj(data_path, "TEXT_test_fmri_top.npy"))
    test_features = np.load(opj(data_path, "test_text_features.npy"))
    train_fmri_top = np.load(opj(data_path, "TEXT_train_fmri_top.npy"))
    train_features = np.load(opj(data_path, "train_text_features.npy"))

    simple_decoding_model = RidgeCV(alphas=[1, 10, 100, 1e3, 1e4])
    simple_decoding_model.fit(backend.asarray(train_fmri_top).to(device),backend.asarray(train_features).to(device))

    #Use the model to predict test set image features
    test_predictions = simple_decoding_model.predict(backend.asarray(test_fmri_top).to(device))




    res = evaluate_and_log(test_predictions.numpy(),test_features,i+1)
    text_results.append(res)
    text_models[subj] = {"model":simple_decoding_model, "test_predictions":test_predictions, "test_features":test_features}



  0%|          | 0/4 [00:00<?, ?it/s]/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 3795 < 10000). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(
 25%|██▌       | 1/4 [00:07<00:21,  7.22s/it]

Top-1 Accuracy: 3.4221, Top-5 Accuracy: 13.9417 
Identification accuracy: 90.2831


/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 3795 < 10000). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(
 50%|█████     | 2/4 [00:15<00:15,  7.78s/it]

Top-1 Accuracy: 2.7883, Top-5 Accuracy: 10.7731 
Identification accuracy: 86.7171


/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 3795 < 10000). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(
 75%|███████▌  | 3/4 [00:22<00:07,  7.55s/it]

Top-1 Accuracy: 2.2814, Top-5 Accuracy: 9.7592 
Identification accuracy: 86.7827


/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 2244 < 10000). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(
100%|██████████| 4/4 [00:25<00:00,  6.36s/it]

Top-1 Accuracy: 1.7131, Top-5 Accuracy: 8.7794 
Identification accuracy: 85.2092


In [81]:
df_text = pd.DataFrame.from_dict(text_results)
df_text.to_csv("/home/matteo/storage/brain_tuning/results_text_linear.csv")
df_text

,Subject,Identification Accuracy (%),ID Accuracy Baseline (%),Top-1 Accuracy (%),Top1 Baseline (%),Top1 Improvement Over Baseline,Top-5 Accuracy (%),Top5 Baseline (%),Top5 Improvement Over Baseline
0,0,90.283112,50,3.422053,0.126743,27.0,13.941698,0.633714,22.0
1,1,86.717106,50,2.788340,0.126743,22.0,10.773131,0.633714,17.0
2,2,86.782730,50,2.281369,0.126743,18.0,9.759189,0.633714,15.4
3,3,85.209216,50,1.713062,0.214133,8.0,8.779443,1.070664,8.2


## Multimodal decoding

In [91]:
image_models["CSI1"]["test_predictions"]

tensor([[ 0.1721,  0.0232,  0.0613,  ...,  0.0726,  0.0768,  0.2222],
        [ 0.0231, -0.1485,  0.1456,  ..., -0.3089,  0.0558, -0.0024],
        [-0.4820,  0.5566, -0.2041,  ...,  0.2479, -0.3731,  0.0402],
        ...,
        [ 0.4915,  0.0975,  0.5601,  ..., -0.1677, -0.0991, -0.1818],
        [-0.3734,  0.1420,  0.0088,  ..., -0.4706, -0.2087, -0.2411],
        [ 0.0461, -0.1658,  0.0062,  ..., -0.2219, -0.2214,  0.2905]])

In [ ]:
i

In [95]:
## concatenate image and text features to perform multimodal decoding

multimodal_results = []

for i,sub in enumerate(["CSI1","CSI2","CSI3","CSI4"]):

    pred_img_features = image_models[sub]["test_predictions"]
    pred_text_features = text_models[sub]["test_predictions"]
    true_img_features = image_models[sub]["test_features"]
    true_text_features = text_models[sub]["test_features"]

    pred_features = np.concatenate([pred_img_features,pred_text_features],1)
    true_features = np.concatenate([true_img_features,true_text_features],1)

    pred_features = torch.tensor(pred_features)
    true_features = torch.tensor(true_features)

    res = evaluate_and_log(pred_features.cpu().numpy(),true_features.cpu().numpy(),i+1)
    multimodal_results.append(res)



Top-1 Accuracy: 7.2243, Top-5 Accuracy: 23.8276 
Identification accuracy: 94.4257
Top-1 Accuracy: 4.9430, Top-5 Accuracy: 17.4905 
Identification accuracy: 91.6094
Top-1 Accuracy: 4.6895, Top-5 Accuracy: 16.6033 
Identification accuracy: 91.7746
Top-1 Accuracy: 3.8544, Top-5 Accuracy: 17.1306 
Identification accuracy: 91.1980


In [96]:
df_multi = pd.DataFrame.from_dict(multimodal_results)
df_multi.to_csv("/home/matteo/storage/brain_tuning/results_multi_linear.csv")
df_multi

,Subject,Identification Accuracy (%),ID Accuracy Baseline (%),Top-1 Accuracy (%),Top1 Baseline (%),Top1 Improvement Over Baseline,Top-5 Accuracy (%),Top5 Baseline (%),Top5 Improvement Over Baseline
0,1,94.425733,50,7.224335,0.126743,57.0,23.827630,0.633714,37.6
1,2,91.609407,50,4.942966,0.126743,39.0,17.490494,0.633714,27.6
2,3,91.774591,50,4.689480,0.126743,37.0,16.603295,0.633714,26.2
3,4,91.198041,50,3.854390,0.214133,18.0,17.130621,1.070664,16.0
